<a href="https://colab.research.google.com/github/Ztumbling/deep-learning-from-scratch/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorboardX

     |████████████████████████████████| 124 kB 7.7 MB/s 


In [2]:
cd ./drive/MyDrive/rmb_try/

/content/drive/MyDrive/rmb_try


In [3]:
import os
import torch
import torch.nn as nn
import random
from torch.utils.data import DataLoader
import imp
import Mydataset
imp.reload(Mydataset)
from Mydataset import RMBdataset
import glob
from torch import optim
from tensorboardX import SummaryWriter

In [10]:
EPOCH = 10
BATCH_SIZE = 4
LR = 0.01
log_interval = 10
val_interval = 1
data_dir = './RMB_data'

In [11]:
#split to 3 datasets
path0 = os.path.join(data_dir,'1','*.jpg')
path1 = os.path.join(data_dir,'100','*.jpg')
jpg_list_0 = glob.glob(path0)
jpg_list_1 = glob.glob(path1)
final_info = []
for i in jpg_list_0:
    final_info.append((i,0))
for j in jpg_list_1:
    final_info.append((j,1))
random.shuffle(final_info)
len_data = len(final_info)

train_info = final_info[0:int(0.8*len_data-1)]
valid_info = final_info[int(0.8*len_data):int(0.9*len_data-1)]
test_info = final_info[int(0.9*len_data):len_data]

train_dataset = RMBdataset(train_info,1)
valid_dataset = RMBdataset(valid_info,0)
test_dataset = RMBdataset(test_info,0)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE)

In [12]:
#create 模型
class LeNet(nn.Module):
    def __init__(self, classes):
        super(LeNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,6,5),
            #nn.BatchNorm2d(num_features=6),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(6,28,3),
            #nn.BatchNorm2d(num_features=28),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(28*6*6,120),
            #nn.BatchNorm1d(num_features=120),
            nn.ReLU(),
            nn.Linear(120,84),
            #nn.BatchNorm1d(num_features=84),
            nn.ReLU(),
            nn.Linear(84,classes)
        )

    def forward(self,x):
        x = self.features(x)
        x = x.view(x.size()[0], -1)
        #x = x.resize(x.size()[0], 28, 6, 6)
        x = self.classifier(x)
        return x

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight.data, 0, 0.1)
                m.bias.data.zero_()
                
net = LeNet(2)

In [13]:
#Loss
criterion = nn.CrossEntropyLoss()  
#Optimizer
optimizer = optim.SGD(net.parameters(),lr = LR,momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1) 

In [ ]:
iter_count = 0
train_curve = list()
valid_curve = list()
for epoch in range(EPOCH):
    net.train()
    loss_mean = 0.
    correct = 0.
    total = 0.
    for i, data in enumerate(train_loader):
        iter_count += 1
        #forward
        inputs, labels = data
        outputs = net(inputs)
        #backward
        optimizer.zero_grad()
        loss = criterion(outputs,labels)
        loss.backward()
        #optimize
        optimizer.step()
        #add up
        _,predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).squeeze().sum().numpy()

        log_dir = "./train_log"
        writer = SummaryWriter(log_dir=log_dir, filename_suffix="12345678")
        writer.add_scalars("Loss", {"Train": loss.item()}, iter_count)
        writer.add_scalars("Accuracy", {"Train": correct / total}, iter_count)
        #        # 打印训练信息
        loss_mean += loss.item()
        train_curve.append(loss.item())
        if (i+1) % log_interval == 0:
            loss_mean = loss_mean / log_interval
            print("Training:Epoch[{:0>3}/{:0>3}] Iteration[{:0>3}/{:0>3}] Loss: {:.4f} Acc:{:.2%}".format(
                epoch+1, EPOCH, i+1, len(train_loader), loss_mean, correct / total))
            loss_mean = 0.
    for name, param in net.named_parameters():
        writer.add_histogram(name + '_grad', param.grad, epoch)
        writer.add_histogram(name + '_data', param, epoch)
    scheduler.step()  # 每个 epoch 更新学习率
    if (epoch+1) % val_interval == 0:
        correct_val = 0.
        total_val = 0.
        loss_val = 0.
        net.eval()
        with torch.no_grad():
            for j, data in enumerate(valid_loader):
                inputs, labels = data
                outputs = net(inputs)
                loss = criterion(outputs, labels)

                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).squeeze().sum().numpy()

                loss_val += loss.item()

            valid_curve.append(loss_val/valid_loader.__len__())
            print("Valid:\t Epoch[{:0>3}/{:0>3}] Iteration[{:0>3}/{:0>3}] Loss: {:.4f} Acc:{:.2%}".format(
                epoch+1, EPOCH, j+1, len(valid_loader), loss_val, correct_val / total_val))


In [20]:
##. TEST THE RESULT ##
net.eval()
test_loader = DataLoader(dataset=test_dataset, batch_size=1)
with torch.no_grad():
    for k, data in enumerate(test_loader):
        inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data,1)
        total_val += labels.size(0)
        correct_val += (predicted == labels).squeeze().sum().numpy()
print('Acc:{:.2%}'.format(correct_val/total_val))

Acc:100.00%


In [ ]:

%reload_ext tensorboard
%tensorboard --logdir=./train_log/

In [15]:
!rm -fr ./train_log/

In [22]:
    lenet = LeNet(classes=2)
    import torchsummary
    print(torchsummary.summary(lenet, (3, 32, 32), device="cpu"))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             456
              ReLU-2            [-1, 6, 28, 28]               0
         MaxPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 28, 12, 12]           1,540
              ReLU-5           [-1, 28, 12, 12]               0
         MaxPool2d-6             [-1, 28, 6, 6]               0
            Linear-7                  [-1, 120]         121,080
              ReLU-8                  [-1, 120]               0
            Linear-9                   [-1, 84]          10,164
             ReLU-10                   [-1, 84]               0
           Linear-11                    [-1, 2]             170
Total params: 133,410
Trainable params: 133,410
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/

In [21]:
print(net)

LeNet(
  (features): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 28, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=1008, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=2, bias=True)
  )
)
